In [7]:
import pandas as pd
import re
import os
import random
import numpy as np
#visual
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from wordcloud import WordCloud
import plotly.express as px

#nlp

from transformers import BertForSequenceClassification, BertConfig, BertTokenizer, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import torch
import emoji
from googletrans import Translator

# from google.colab import files
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler, TensorDataset, random_split
from tqdm import tqdm
import statistics
import time
import datetime

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jmspa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
import pandas as pd
combined_df = pd.read_excel('fix_fixed.xlsx')

In [9]:
combined_df

,Data,Label,translated,Unnamed: 3,review_text_cleaned
0,bangka blm bukan kantor pos ambil nya,1,bangka blm bukan kantor pos ambil nya,NaN,bangka belum bukan kantor pos ambil nya
1,halo min apa kpr sudah berjalan bisa minta di...,2,lingkaran cahaya min apa kpr sudah berjalan b...,NaN,lingkaran cahaya min apa kpr sudah berjalan bi...
2,perancis,1,perancis,NaN,perancis
3,kapan giveaway nya nih megamin [smirking face],0,kapan memberi secara gratis nya nih megamin w...,NaN,kapan memberi secara gratis nya ini megamin wa...
4,geh yang balam tau gak soto tangkar yang dulu ...,1,geh yang balam tau gak soto tangkar yang dulu...,NaN,geh yang balam tahu tidak soto tangkar yang du...
...,...,...,...,...,...
31994,kak brarti perlu download welma lagi kan utk b...,0,kak brarti perlu unduh welma lagi kan utk beli...,NaN,kak berarti perlu unduh welma lagi kan untuk b...
31995,semoga kita terhindar dari modus penipuan bri ...,0,semoga kita terhindar dari modus penipuan bri ...,NaN,semoga kita terhindar dari modus penipuan bri ...
31996,hah dimana tuh tahun kemaren ngajuin mintanya...,2,ha dimana tuh tahun kemaren ngajuin mintanya 2jt,NaN,tertawa di mana itu tahun kemarin ngajuin mint...
31997,masang nomor mudah mudahan aja bisa tembus tuh...,0,masang nomor mudah mudahan aja bisa tembus tuh...,NaN,memasang nomor mudah mudahan saja bisa tembus ...


In [11]:
combined_df['review_text_cleaned'] = combined_df['review_text_cleaned'].astype(str)

In [4]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-large-p2")
config = BertConfig.from_pretrained("indobenchmark/indobert-large-p2")
config.num_labels = 3

# Instantiate model
model = BertForSequenceClassification.from_pretrained("indobenchmark/indobert-large-p2", config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import statistics
sent_length = []

# For every sentence...
for sent in combined_df['review_text_cleaned']:
    # print(sent)
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    sent_length.append(len(input_ids))
    if len(input_ids) == 1265:
        print(sent)

print('Average length = ', sum(sent_length)/len(sent_length))
print('Median length = ', statistics.median(sent_length))
print('Max lenght =', max(sent_length))
print('Min lenght = ', min(sent_length))

Average length =  17.883246351448484
Median length =  14
Max lenght = 864
Min lenght =  3


In [13]:
# Tokenize all of the sentences and map the tokens to their word IDs.
import torch
input_ids = []
attention_masks = []

# For every sentence...
for sent in combined_df['review_text_cleaned']:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 24,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',# Return pytorch tensors.
                   )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(combined_df['Label'])

print(input_ids)
print(attention_masks)
print(labels)
# Print sentence 0, now as a list of IDs.
print('Original: ', combined_df['translated'][0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\jmspa\anaconda3\envs\coba_GRU\Lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


tensor([[   2, 9180,  659,  ...,    0,    0,    0],
        [   2, 7738, 3729,  ...,   41, 4189,    3],
        [   2, 6057,    3,  ...,    0,    0,    0],
        ...,
        [   2, 7519,   26,  ...,    0,    0,    0],
        [   2, 5366, 1288,  ...,    0,    0,    0],
        [   2, 1866, 3121,  ...,  629, 3588,    3]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])
tensor([1, 2, 1,  ..., 2, 0, 2])
Original:   bangka blm bukan kantor pos ambil nya
Token IDs: tensor([   2, 9180,  659,  531, 1571,  598, 3422, 1107,    3,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0])


In [14]:
with torch.no_grad():
    last_hidden_states = model(input_ids)[0] # Models outputs are now tuples
last_hidden_states = last_hidden_states.mean(1)
print(last_hidden_states)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [ ]:
from sklearn.model_selection import train_test_split
X=last_hidden_states
y=combined_df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=30)

In [ ]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB,BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

X = last_hidden_states.numpy().reshape(-1, 1)

num_bins = 10
X_discretized = np.digitize(X, bins=np.linspace(X.min(), X.max(), num_bins))

# Extract the labels from the DataFrame
y = combined_df['Label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_discretized, y, test_size=0.2, random_state=42)

# Train the Naive Bayes classifier on the discretized embeddings
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6443661971830986


In [ ]:
nb_classifier = ComplementNB()
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.2112676056338028


In [ ]:
nb_classifier = BernoulliNB()
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6443661971830986
